In [81]:
import numpy as np
import pandas as pd

In [82]:
apple = pd.read_csv("AAPL.csv")
amazon = pd.read_csv("AMZN.csv")
google = pd.read_csv("GOOG.csv")
google_l = pd.read_csv("GOOGL.csv")
microsoft = pd.read_csv("MSFT.csv")
tesla = pd.read_csv("TSLA.csv")

In [83]:
def paint(data):
    data = data.iloc[::-1]
    data = data.reset_index(drop=True)
    data['Label'] = np.where(data['adj close'].shift(-1) > data['adj close'], 1, 0)
    #data['Label'] = (data['close'] > data['open']).astype(int)
    return data
apple = paint(apple)
amazon = paint(amazon)
google = paint(google)
google_l = paint(google_l)
microsoft = paint(microsoft)
tesla = paint(tesla)

In [84]:
tweets = pd.read_csv("D:\johnny的文件夹\FDU 课程\大二下\大数据商务分析\Final project\data\Tweet\Tweet.csv")
company_tweets = pd.read_csv(
    "D:\johnny的文件夹\FDU 课程\大二下\大数据商务分析\Final project\data\Tweet\Company_Tweet.csv")
tweets.drop('writer', axis=1, inplace=True)
df_tweets = pd.merge(tweets, company_tweets, on='tweet_id', how='left')
df_tweets['post_date'] = pd.to_datetime(df_tweets['post_date'], unit='s')
df_tweets['post_date'] = df_tweets['post_date'].dt.strftime('%Y-%m-%d')
df_tweets.drop('tweet_id', axis=1, inplace=True)

In [85]:
df_tweets['tweet_score'] = 1 * df_tweets['comment_num'] + 1 * df_tweets['retweet_num'] + 1.2 * df_tweets['like_num']

In [86]:
df_tweets = df_tweets[df_tweets['tweet_score'] >= 50].reset_index()

In [87]:
df_tweets.drop('index', axis=1, inplace=True)
df_tweets.drop('comment_num', axis=1, inplace=True)
df_tweets.drop('retweet_num', axis=1, inplace=True)
df_tweets.drop('like_num', axis=1, inplace=True)


In [88]:
df_tweets.rename(columns={'post_date': 'date'}, inplace=True)
df_tweets['date'] = pd.to_datetime(df_tweets['date'])
df_tweets

,date,body,ticker_symbol,tweet_score
0,2015-01-01,Jeff Bezos lost $7.4 billion in Amazon's worst...,AMZN,228.4
1,2015-01-01,Jeff Bezos lost $7.4 billion in Amazon's worst...,AMZN,198.4
2,2015-01-01,Analyst Report on Top Stock Market Options for...,AAPL,433.6
3,2015-01-01,Analyst Report on Top Stock Market Options for...,AMZN,433.6
4,2015-01-01,Top Stock Analyst Releases Report on Top Stock...,AAPL,433.4
...,...,...,...,...
50038,2019-12-31,"Dumb question, is this enough people to run a ...",TSLA,65.8
50039,2019-12-31,My top picks for 2020 (in no particular order)...,TSLA,98.2
50040,2019-12-31,Ima say this now and ima say it loud.GAPUSHOME...,AMZN,73.6
50041,2019-12-31,"Hey @ChrisSpelter, does this mean $TSLA is wor...",TSLA,53.6


In [89]:
def explore(df,company):
    df1 = df[df['ticker_symbol'] == company]
    score = df1.groupby('date')['tweet_score'].sum()
    df1 = pd.concat([score],axis=1)
    df1 = df1.reset_index()
    return df1
apple_tweets = explore(df_tweets,'AAPL')
amazon_tweets = explore(df_tweets,'AMZN')
google_tweets = explore(df_tweets,'GOOG')
googlel_tweets = explore(df_tweets,'GOOGL')
microsoft_tweets = explore(df_tweets,'MSFT')
tesla_tweets = explore(df_tweets,'TSLA')

In [90]:
apple_tweets

,date,tweet_score
0,2015-01-01,1304.2
1,2015-01-02,1687.2
2,2015-01-04,481.8
3,2015-01-05,280.0
4,2015-01-08,248.2
...,...,...
1498,2019-12-27,202.6
1499,2019-12-28,1696.4
1500,2019-12-29,467.8
1501,2019-12-30,1468.4


In [91]:
def make_stock(stock,tweet):
    stock['date'] = pd.to_datetime(stock['date'])
    tweet['date']= pd.to_datetime(tweet['date'])
    
    start_date = '2015-01-01'
    end_date = '2019-12-31'
    stock = stock.loc[(stock['date'] >= start_date) & (stock['date'] <= end_date)]
    result = pd.merge(stock,tweet,on='date',how='left')
    return result
apple = make_stock(apple,apple_tweets).dropna()
amazon = make_stock(amazon,amazon_tweets).dropna()
google = make_stock(google,google_tweets).dropna()
googlel = make_stock(google_l,googlel_tweets).dropna()
microsoft = make_stock(microsoft,microsoft_tweets).dropna()
tesla = make_stock(tesla,tesla_tweets).dropna()

In [92]:
apple

,date,open,high,low,close,adj close,volume,Label,tweet_score
0,2015-01-02,111.389999,111.440002,107.349998,109.330002,100.216454,53204600,0,1687.2
1,2015-01-05,108.290001,108.650002,105.410004,106.250000,97.393181,64285500,1,280.0
4,2015-01-08,109.230003,112.150002,108.699997,111.889999,102.563072,59364500,1,248.2
5,2015-01-09,112.669998,113.250000,110.209999,112.010002,102.673050,53699500,0,93.8
6,2015-01-12,112.599998,112.629997,108.800003,109.250000,100.143120,49650800,1,76.6
...,...,...,...,...,...,...,...,...,...
1253,2019-12-24,284.690002,284.890015,282.920013,284.269989,283.596924,12119700,1,572.8
1254,2019-12-26,284.820007,289.980011,284.700012,289.910004,289.223602,23280300,0,853.4
1255,2019-12-27,291.119995,293.970001,288.119995,289.799988,289.113831,36566500,1,202.6
1256,2019-12-30,289.459991,292.690002,285.220001,291.519989,290.829773,36028600,1,1468.4


In [93]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV

def lgr_model(stock):
    data_X = stock[['open','high','low','close', 'volume','tweet_score']]
    data_y = stock['Label']
    X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=4)
    model = LogisticRegression(max_iter=20000)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    print("Precision:{}, Recall:{}, f1:{}".format(precision, recall, f1))

    
    pred = model.predict(data_X)
    return (model,pred)


In [94]:
prediction_apple = lgr_model(apple)[1]
apple['pre_Label'] = prediction_apple

Precision:0.5112107623318386, Recall:1.0, f1:0.6765578635014837


In [95]:
apple.reset_index(inplace=True)
initial_fund = 10000
funds = initial_fund
for index in range(len(apple) - 1):
    if apple.loc[index, 'pre_Label'] == 1:
        funds += (apple.loc[index + 1, 'close'] - apple.loc[index + 1, 'open'])
print(funds)

10103.199996948242


In [96]:
prediction_amazon = lgr_model(amazon)

Precision:0.553072625698324, Recall:0.99, f1:0.7096774193548387


In [97]:
amazon['pre_Label'] = prediction_amazon[1]
#amazon.drop('level_0')

In [98]:
amazon.reset_index(inplace=True)
initial_fund = 10000
funds = initial_fund
for index in range(len(amazon) - 1):
    if amazon.loc[index, 'pre_Label'] == 1:
        funds += (amazon.loc[index + 1, 'close'] - amazon.loc[index + 1, 'open'])
print(funds)

9231.730346679688


In [99]:
prediction_google = lgr_model(google)

Precision:0.5972222222222222, Recall:0.6825396825396826, f1:0.6370370370370371


In [100]:
google['pre_Label'] = prediction_google[1]
#amazon.drop('level_0')
google.reset_index(inplace=True)
initial_fund = 10000
funds = initial_fund
for index in range(len(google) - 1):
    if google.loc[index, 'pre_Label'] == 1:
        funds += (google.loc[index + 1, 'close'] - google.loc[index + 1, 'open'])
print(funds)

10400.369995117188


In [101]:
prediction_googlel = lgr_model(googlel)

Precision:0.5851063829787234, Recall:0.7333333333333333, f1:0.650887573964497


In [102]:
googlel['pre_Label'] = prediction_googlel[1]
#amazon.drop('level_0')
googlel.reset_index(inplace=True)
initial_fund = 10000
funds = initial_fund
for index in range(len(googlel) - 1):
    if googlel.loc[index, 'pre_Label'] == 1:
        funds += (googlel.loc[index + 1, 'close'] - googlel.loc[index + 1, 'open'])
print(funds)

10162.990234375


In [103]:
prediction_microsoft = lgr_model(microsoft)

Precision:0.5877192982456141, Recall:1.0, f1:0.7403314917127072


In [104]:
microsoft['pre_Label'] = prediction_microsoft[1]
#amazon.drop('level_0')
microsoft.reset_index(inplace=True)
initial_fund = 10000
funds = initial_fund
for index in range(len(microsoft) - 1):
    if microsoft.loc[index, 'pre_Label'] == 1:
        funds += (microsoft.loc[index + 1, 'close'] - microsoft.loc[index + 1, 'open'])
print(funds)

10016.829956054688


In [105]:
prediction_tesla = lgr_model(tesla)

Precision:0.4606741573033708, Recall:0.44565217391304346, f1:0.4530386740331492


In [106]:
tesla['pre_Label'] = prediction_tesla[1]
#amazon.drop('level_0')
tesla.reset_index(inplace=True)
initial_fund = 10000
funds = initial_fund
for index in range(len(tesla) - 1):
    if tesla.loc[index, 'pre_Label'] == 1:
        funds += (tesla.loc[index + 1, 'close'] - tesla.loc[index + 1, 'open'])
print(funds)

10141.149795532227


In [107]:
from sklearn.svm import SVC
def svm_model(stock):
    X = stock[['open','high','low','adj close', 'volume','tweet_score']]
    y = stock['Label']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    model = SVC(class_weight='balanced', random_state=4)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    #print(y_pred)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print("Precision:{}, Recall:{}, f1:{}".format(precision, recall, f1))
    
    param_grid = {
    'C': [400,500,600,700],
    'gamma': ['scale', 'auto',0.01,0.1,1],
    'kernel': ['rbf']
}
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='precision')
    grid_search.fit(X_train, y_train)
    
    print("Best parameters found:", grid_search.best_params_)
    
    best_svm_model = grid_search.best_estimator_
    best_svm_model.fit(X_train, y_train)
    
    y_pred_best = best_svm_model.predict(X_test)
    precision_best = precision_score(y_test, y_pred_best)
    recall_best = recall_score(y_test, y_pred_best)
    f1_best = f1_score(y_test, y_pred_best)
    print(precision_best, recall_best, f1_best)
    
    pred = model.predict(X_scaled)
    print(pred)
    
    return (model,pred)


In [108]:
def fund(stock):
    stock.reset_index(inplace=True)
    initial_fund = 10000
    funds = initial_fund
    for index in range(len(stock) - 1):
        if stock.loc[index, 'pre_Label1'] == 1:
            funds += (stock.loc[index + 1, 'close'] - stock.loc[index + 1, 'open'])
    print(funds)

In [109]:
prediction_apple = svm_model(apple)[1]
apple['pre_Label1'] = prediction_apple

Precision:0.5109489051094891, Recall:0.6086956521739131, f1:0.5555555555555556
Best parameters found: {'C': 500, 'gamma': 1, 'kernel': 'rbf'}
0.5398230088495575 0.5304347826086957 0.5350877192982456
[0 0 0 ... 1 1 1]


In [110]:
fund(apple)

10141.01000213623


In [111]:
prediction_amazon = svm_model(amazon)[1]
amazon['pre_Label1'] = prediction_amazon
fund(amazon)

Precision:0.581081081081081, Recall:0.4215686274509804, f1:0.48863636363636365
Best parameters found: {'C': 400, 'gamma': 0.01, 'kernel': 'rbf'}
0.5894736842105263 0.5490196078431373 0.5685279187817259
[0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1
 1 1 0 1 0 0 0 0 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 1 0 0 0 1
 0 0 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 0 0
 0 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0

In [112]:
prediction_google = svm_model(google)[1]

Precision:0.6527777777777778, Recall:0.7230769230769231, f1:0.6861313868613139
Best parameters found: {'C': 600, 'gamma': 0.01, 'kernel': 'rbf'}
0.6172839506172839 0.7692307692307693 0.684931506849315
[1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1
 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 1 1 0 0 1 1 1 0 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 

In [113]:
google['pre_Label1'] = prediction_google
google.reset_index(inplace=True)

In [114]:
initial_fund = 10000
funds = initial_fund
for index in range(len(google) - 1):
    if google.loc[index, 'pre_Label1'] == 1:
        funds += (google.loc[index + 1, 'close'] - google.loc[index + 1, 'open'])
print(funds)

10197.001342773438


In [115]:
prediction_googlel = svm_model(googlel)[1]

Precision:0.6351351351351351, Recall:0.6438356164383562, f1:0.6394557823129252
Best parameters found: {'C': 400, 'gamma': 0.1, 'kernel': 'rbf'}
0.5409836065573771 0.4520547945205479 0.4925373134328358
[0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1
 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0
 1 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0
 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 1 0 1
 1 1 1 1 1 1 1 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 

In [116]:
googlel['pre_Label'] = prediction_googlel
googlel.reset_index(inplace=True)
initial_fund = 10000
funds = initial_fund
for index in range(len(googlel) - 1):
    if googlel.loc[index, 'pre_Label'] == 1:
        funds += (googlel.loc[index + 1, 'close'] - googlel.loc[index + 1, 'open'])
print(funds)

10247.270263671875


In [117]:
prediction_microsoft = svm_model(microsoft)[1]

Precision:0.5384615384615384, Recall:0.5645161290322581, f1:0.5511811023622047
Best parameters found: {'C': 700, 'gamma': 1, 'kernel': 'rbf'}
0.5230769230769231 0.5483870967741935 0.5354330708661418
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0
 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1
 0 1 0 0 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 0 1 1 1 1 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 0
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 0 

In [118]:
microsoft['pre_Label'] = prediction_microsoft
microsoft.reset_index(inplace=True)
initial_fund = 10000
funds = initial_fund
for index in range(len(microsoft) - 1):
    if microsoft.loc[index, 'pre_Label'] == 1:
        funds += (microsoft.loc[index + 1, 'close'] - microsoft.loc[index + 1, 'open'])
print(funds)

10017.109943389893


In [119]:
prediction_tesla = svm_model(tesla)[1]
tesla['pre_Label1'] = prediction_tesla
fund(tesla)

Precision:0.5121951219512195, Recall:0.23076923076923078, f1:0.3181818181818182
Best parameters found: {'C': 700, 'gamma': 1, 'kernel': 'rbf'}
0.5 0.5164835164835165 0.5081081081081081
[0 1 1 1 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1
 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 